# Gas Station Simulator Extension: Requesting Multiple Resources

#### _Edward Krueger, edkrueger@gmail.com_

## Purpose

Show how to have a process request multiple resources in simpy. This is not a default default behavior, so we'll require a work around.

## Overview
We want to add a behavior to Alice's simulator where some cars take up multiple pumps (whether because their cars a very large, because they lack awareness or because they are towing a boat.)

## Simplifying Assumptions

To highlight the behavior we're looking at we'll make some simplifying assumptions. We'll simplify the simulation so that there are a fixed two pumps, which we'll model and report metrics on directly. We'll also make the assumption that cars are assigned randomly to a pump when they arrive at the gas station. (Perhaps some cars require diesel and some require gasoline and each pump only has one type of fuel.) Finally, we'll assume that when a car block both pumps, it always pumps the fuel from the first one. (Perhaps the large cars are all diesel trucks and they block both pumps, but always fill up with the diesel.) Finally, we'll assume that the utilization metric counts for all blocked pumps whether they are pumping or not.

# Gas Station Simulator Solution

## Setup

In [1]:
import simpy

import numpy as np

from scipy.stats import expon
from scipy.stats import norm

from contextlib import ExitStack

In [2]:
class GasPump():
    
    """
    Models a gas pump as a simpy.Resource.
    
    Arguments:
    -----------
    name (str): The name or ID of a pump
    env (simpy.Environment): The simpy environment
    """
    
    def __init__(self, name, env):
        
        self.name = name
                
        self.env = env
        
        self.resource = simpy.PriorityResource(self.env, capacity=1)
        
        self.minutes_utilized = 0
        self.gallons_sold = 0

In [3]:
class Car():

    """
    Models a car.
    The car's simulation process in the simulation is the method .run().
    
    Arguments:
    -----------
    id (int): A unique id for the car
    gas_required (float): Amount of gallons a car will fill
    lay_time (float): Extra minutes that the car stays at the pump
    gas_pump (GasStation): The gas pumps the car is assigned to.
    env (simpy.Environment): The simpy environment
    
    """
    
    def __init__(
        self,
        id,
        gas_required,
        lay_time,
        gas_pumps,
        env
    ):
        
        self.id = id
        self.gas_required = gas_required
        self.lay_time = lay_time
        self.gas_pumps = gas_pumps
        self.env = env
        
        self.wait_time = np.inf
        self.finished = False
        
        self.pump_names = " ".join([gas_pump.name for gas_pump in gas_pumps])
        
        self.action = self.env.process(self.run())
        
    def run(self):
        
        queue_time = self.env.now
        
        print(f'Car {self.id} arrives at {round(queue_time, 2)} minutes')
        
                
        with ExitStack() as stack:
            
            reqs = [
                stack.enter_context(
                    res.resource.request(
                        priority=1
                    )
                ) for res in self.gas_pumps
            ]
            
            yield self.env.all_of(reqs)
            
            pump_start_time = self.env.now
            
            print(f'Car {self.id} begins utilizing pumps {self.pump_names} at {round(pump_start_time, 2)} minutes')

            pump_time = self.gas_required / PUMP_RATE
            utilization = max(pump_time, self.lay_time)
            
            yield self.env.timeout(utilization)
            
            pump_end_time = self.env.now
            
            print(f'Car {self.id} end utilizing pumps {self.pump_names} at {round(pump_end_time, 2)} minutes')
            
            self.finished = True
            
            self.wait_time = pump_start_time - queue_time
            
            # add the utilization to all pumps
            for gas_pump in self.gas_pumps:
                gas_pump.minutes_utilized += utilization
                
            # add the gas to only the first pump
            self.gas_pumps[0].gallons_sold += self.gas_required

In [4]:
class ClosureEvent():
    
    def __init__(
        self,
        start_time,
        duration,
        pump,
        env
    ):
        
        self.start_time = start_time
        self.duration = duration
        self.pump = pump
        
        self.env = env
        
        self.action = self.env.process(self.run())
        
    def run(self):
        
        with self.pump.resource.request(priority=0) as req:
            
            yield req
            
            print(f'Closure event starts at pump {self.pump.name} starts at {self.env.now}')
            
            remaining_duration = max([
                0,
                self.duration - (self.env.now - self.start_time)
            ])
            
            yield self.env.timeout(remaining_duration)
            
            print(print(f'Closure event ends at pump {self.pump.name} starts at {self.env.now}'))
    

In [5]:
class GlobalClosureEvent():
    
    def __init__(
        self,
        start_time,
        duration,
        env
    ):
        
        self.start_time = start_time
        self.duration = duration
        
        self.env = env
        
        self.action = self.env.process(self.run())
    
    def run(self):
        
        for pump in pumps:

            ClosureEvent(
                start_time=self.start_time,
                duration=self.duration,
                pump=pump,
                env=self.env
            )
            
        yield env.timeout(0)

In [6]:
def closure_scheduler(env):
    
    yield env.timeout(50)
    
    GlobalClosureEvent(
        start_time=50,
        duration=30,
        env=env
    )

In [7]:
def scheduler(env):
    
    """
    Controls the arrival of cars at the gas station.
    
    Arguments:
    -----------
    env (simpy.Environment): The simpy environment
    """
    
    id = 0
    
    while True:
        
        waiting_time = expon.rvs(loc=0, scale=EXPECTED_WAIT)
        
        std_norm = norm.rvs()
        gas_required = std_norm * GAS_REQUIRED_STD + GAS_REQUIRED_MEAN
        gas_required = max([0, gas_required])
        
        lay_time = expon.rvs(loc=0, scale=EXPECTED_LAY_TIME)
        
        yield env.timeout(waiting_time)
        
        gas_pumps = np.random.choice(
            a = [
                [np.random.choice(pumps)],
                pumps
            ],
            p = [
                1- PROP_LARGE_CARS,
                PROP_LARGE_CARS,
            ]
        )
        
        CARS.append(
            Car(
                id=id,
                gas_required=gas_required,
                lay_time=lay_time,
                gas_pumps=gas_pumps,
                env=env
            )
        )
        
        id += 1

## Input parameters and run the simulation

In [8]:
EXPECTED_WAIT = 2 # in minutes

GAS_REQUIRED_MEAN = 15 # in gallons
GAS_REQUIRED_STD = 5 # in gallons

PUMP_RATE = 10 # in gallons per minute

EXPECTED_LAY_TIME = 5 # in minutes

PROP_LARGE_CARS = .15 # in [0,1]

sim_time = 24 * 60 # in minutes

## Wrap up the simulation and metrics

In [9]:
env = simpy.Environment()
CARS = []

pump_1 = GasPump("1", env)
pump_2 = GasPump("2", env)
pumps = [pump_1, pump_2]

env.process(scheduler(env))
env.process(closure_scheduler(env))

env.run(until=sim_time)


Car 0 arrives at 0.21 minutes
Car 0 begins utilizing pumps 2 at 0.21 minutes
Car 1 arrives at 0.36 minutes
Car 2 arrives at 0.7 minutes
Car 0 end utilizing pumps 2 at 3.86 minutes
Car 1 begins utilizing pumps 2 at 3.86 minutes
Car 3 arrives at 5.98 minutes
Car 3 begins utilizing pumps 1 at 5.98 minutes
Car 4 arrives at 6.38 minutes
Car 5 arrives at 7.21 minutes
Car 6 arrives at 7.29 minutes
Car 7 arrives at 8.03 minutes
Car 8 arrives at 9.33 minutes
Car 9 arrives at 9.92 minutes
Car 10 arrives at 10.24 minutes
Car 1 end utilizing pumps 2 at 10.73 minutes
Car 2 begins utilizing pumps 2 at 10.73 minutes
Car 11 arrives at 12.14 minutes
Car 12 arrives at 12.98 minutes
Car 13 arrives at 13.25 minutes
Car 2 end utilizing pumps 2 at 14.27 minutes
Car 14 arrives at 16.31 minutes
Car 3 end utilizing pumps 1 at 17.92 minutes
Car 4 begins utilizing pumps 1 2 at 17.92 minutes
Car 15 arrives at 19.71 minutes
Car 16 arrives at 29.82 minutes
Car 17 arrives at 29.88 minutes
Car 18 arrives at 30.13 min

Car 331 arrives at 703.98 minutes
Car 332 arrives at 704.12 minutes
Car 176 end utilizing pumps 1 at 708.26 minutes
Car 177 begins utilizing pumps 1 at 708.26 minutes
Car 333 arrives at 708.85 minutes
Car 334 arrives at 709.79 minutes
Car 335 arrives at 710.93 minutes
Car 177 end utilizing pumps 1 at 711.68 minutes
Car 178 begins utilizing pumps 1 at 711.68 minutes
Car 336 arrives at 711.95 minutes
Car 337 arrives at 713.69 minutes
Car 338 arrives at 714.96 minutes
Car 178 end utilizing pumps 1 at 715.05 minutes
Car 180 begins utilizing pumps 1 2 at 715.05 minutes
Car 180 end utilizing pumps 1 2 at 716.38 minutes
Car 184 begins utilizing pumps 2 at 716.38 minutes
Car 181 begins utilizing pumps 1 at 716.38 minutes
Car 184 end utilizing pumps 2 at 717.35 minutes
Car 185 begins utilizing pumps 2 at 717.35 minutes
Car 339 arrives at 718.44 minutes
Car 181 end utilizing pumps 1 at 718.49 minutes
Car 182 begins utilizing pumps 1 at 718.49 minutes
Car 340 arrives at 721.91 minutes
Car 341 arr

Car 648 arrives at 1377.63 minutes
Car 366 end utilizing pumps 2 at 1381.74 minutes
Car 367 begins utilizing pumps 1 2 at 1381.74 minutes
Car 649 arrives at 1381.86 minutes
Car 650 arrives at 1382.86 minutes
Car 651 arrives at 1386.49 minutes
Car 652 arrives at 1386.93 minutes
Car 653 arrives at 1387.86 minutes
Car 654 arrives at 1388.77 minutes
Car 655 arrives at 1388.82 minutes
Car 367 end utilizing pumps 1 2 at 1392.5 minutes
Car 369 begins utilizing pumps 2 at 1392.5 minutes
Car 368 begins utilizing pumps 1 at 1392.5 minutes
Car 656 arrives at 1393.15 minutes
Car 369 end utilizing pumps 2 at 1393.52 minutes
Car 371 begins utilizing pumps 2 at 1393.52 minutes
Car 657 arrives at 1393.69 minutes
Car 371 end utilizing pumps 2 at 1395.05 minutes
Car 368 end utilizing pumps 1 at 1396.98 minutes
Car 370 begins utilizing pumps 1 at 1396.98 minutes
Car 658 arrives at 1397.74 minutes
Car 659 arrives at 1400.76 minutes
Car 660 arrives at 1402.37 minutes
Car 661 arrives at 1403.12 minutes
Car 

In [10]:
avg_wait_time = np.array([car.wait_time for car in CARS if car.finished]).mean()
total_gallons_sold = sum([pump.gallons_sold for pump in pumps])

for pump in pumps:
    utilization = pump.minutes_utilized / sim_time * len(pumps) 
    print(f'Utilization for pump {pump.name}: {utilization}')
    
for pump in pumps:
    gallons = pump.gallons_sold
    print(f'Gallons sold at pump {pump.name}: {gallons}')
    


print(f'Average wait time (minutes): {avg_wait_time}')
print(f'Total Gallons Sold: {total_gallons_sold}')

Utilization for pump 1: 1.7328734117308335
Utilization for pump 2: 1.3780089968571283
Gallons sold at pump 1: 3484.2239240663566
Gallons sold at pump 2: 2442.3457501210028
Average wait time (minutes): 325.0779479214482
Total Gallons Sold: 5926.56967418736
